In [1]:
import networkx as nx
import numpy as np
import random
import osmnx as ox
import folium
from folium.plugins import MarkerCluster
import webbrowser
import requests
from itertools import permutations

In [2]:
API_KEY = '5b3ce3597851110001cf62481a74eda827a04828adf4f255bf046343'

In [3]:
z = 5
ville = "Wazemmes, FR"
G_ox = ox.graph_from_place(ville, network_type="drive")
ville_node = list(G_ox.nodes)
ville_edge = list(G_ox.edges)

points = [(G_ox.nodes[node]['y'], G_ox.nodes[node]['x']) for node in ville_node]

edge_list = [((G_ox.nodes[edge[0]]['y'], G_ox.nodes[edge[0]]['x']),
              (G_ox.nodes[edge[1]]['y'], G_ox.nodes[edge[1]]['x']),
              G_ox.edges[edge]['length'])
             for edge in ville_edge]

selected_nodes = random.sample(ville_node, min(z, len(ville_node)))

coord_selected_nodes = [(G_ox.nodes[node]['y'], G_ox.nodes[node]['x']) for node in selected_nodes]

In [4]:
# Déclaration des fonctions, distance, elevation , adresse, folium

def distance_time(origin, destination, api_key):
    start_coords = f'{origin[1]},{origin[0]}'
    end_coords = f'{destination[1]},{destination[0]}'
    url = f'https://api.openrouteservice.org/v2/directions/driving-car?api_key={api_key}&start={start_coords}&end={end_coords}'

    try:
        response = requests.get(url)
        response.raise_for_status()  # Vérifier si la requête a réussi

        data = response.json()
        distance = data['features'][0]['properties']['segments'][0]['distance']
        duration = data['features'][0]['properties']['segments'][0]['duration']
        return distance, duration

    except requests.exceptions.RequestException as e:
        print(f"Une erreur s'est produite lors de la requête : {e}")
        return None, None

def calculate_total_distance(path, coordinates):
    total_distance = 0
    total_duration = 0

    for i in range(len(path) - 1):
        origin_index = path[i]
        destination_index = path[i + 1]
        origin = coordinates[origin_index]
        destination = coordinates[destination_index]
        distance, duration = distance_time(origin, destination, API_KEY)
        total_distance += distance
        total_duration += duration

    return total_distance, total_duration

def get_elevation(coordinates):
    base_url = "https://api.open-elevation.com/api/v1/lookup?locations="

    # Définir la taille maximale de chaque sous-liste
    chunk_size = 50  # Vous pouvez ajuster cette valeur en fonction de vos besoins

    # Diviser la liste en sous-listes de taille chunk_size
    chunks = [coordinates[i:i + chunk_size] for i in range(0, len(coordinates), chunk_size)]

    elevations = []

    # Parcourir chaque sous-liste et effectuer la requête
    for chunk in chunks:
        locations = '|'.join([f"{lat},{lon}" for lat, lon in chunk])
        url = f"{base_url}{locations}"

        try:
            response = requests.get(url)
            if response.status_code == 200:
                data = response.json()
                chunk_elevations = [result['elevation'] for result in data['results']]
                elevations.extend(chunk_elevations)
            else:
                return f"Error: {response.status_code}"
        except Exception as e:
            return f"An error occurred: {e}"

    # Renvoyer simplement la liste des élévations
    return elevations

def get_addresses_for_points(points):
    addresses = []

    # Si la liste contient un seul point, convertissez-le en une liste pour un traitement uniforme
    if len(points) == 1:
        points = [points[0]]

    for point in points:
        lat, lon = point

        # Construire l'URL de requête avec les coordonnées
        url = f'https://nominatim.openstreetmap.org/reverse?format=json&lat={lat}&lon={lon}&zoom=18&addressdetails=1'

        try:
            # Effectuer la requête HTTP
            response = requests.get(url)
            response.raise_for_status()  # Vérifier si la requête a réussi

            # Analyser la réponse JSON
            data = response.json()

            # Récupérer le display_name
            display_name = data.get('display_name', 'Adresse non trouvée')
            addresses.append(display_name)

        except requests.exceptions.RequestException as e:
            print(f"Une erreur s'est produite lors de la requête : {e}")
            addresses.append(None)

    return addresses

def create_folium_map_with_points(graph, nodes, elevations, addresses):
    def get_bounds(points):
        lats, lons = zip(*points)
        return [(min(lats), min(lons)), (max(lats), max(lons))]

    # Créer une carte centrée sur le premier point
    map_with_points = folium.Map(location=nodes[0], zoom_start=10)

    # Ajouter des marqueurs pour chaque point avec des informations personnalisées
    marker_cluster = MarkerCluster(max_cluster_radius=20).add_to(map_with_points)

    for i, (node, elevation, address) in enumerate(zip(nodes, elevations, addresses)):
        popup_content = (
            f"<b>Adresse :</b> {address}<br>"
            f"<b>Coords :</b> {node}<br>"
            f"<b>Élévation :</b> {elevation} meters"
        )
        folium.Marker(location=node, popup=popup_content).add_to(marker_cluster)

    # Ajuster le niveau de zoom pour inclure tous les points
    bounds = get_bounds(nodes)
    map_with_points.fit_bounds(bounds)

    # Sauvegarder la carte avec les points ajoutés
    map_with_points.save('map_with_selected_points.html')

    # Ouvrir le fichier HTML dans un navigateur web
    webbrowser.open('map_with_selected_points.html')


In [5]:
create_folium_map_with_points(G_ox, coord_selected_nodes, get_elevation(coord_selected_nodes), get_addresses_for_points(coord_selected_nodes))


In [6]:
# Fonction 2-OPT
def two_opt(path, coordinates, distances_matrix):
    n = len(path)
    best_path = path
    improved = True

    while improved:
        improved = False

        for i in range(1, n - 2):
            for j in range(i + 1, n):
                if j - i == 1:
                    continue  # No reverse for adjacent nodes

                new_path = path.copy()
                new_path[i:j] = reversed(path[i:j])

                current_distance = sum(distances_matrix[path[k - 1]][path[k]] for k in range(n))
                new_distance = sum(distances_matrix[new_path[k - 1]][new_path[k]] for k in range(n))

                if new_distance < current_distance:
                    path = new_path
                    improved = True

    return path

# Créer une matrice des distances pour tous les points avec les nouvelles fonctions
distances_matrix = np.zeros((z, z), dtype=int)

for i in range(z):
    for j in range(z):
        if i != j:
            origin = coord_selected_nodes[i]
            destination = coord_selected_nodes[j]
            distance_time_result = distance_time(origin, destination, API_KEY)
            
            # Check if the distance_time function returned a valid result
            if distance_time_result is not None:
                distance, _ = distance_time_result
                distances_matrix[i][j] = distance
            else:
                # Handle the case where distance_time returned None
                print(f"Error: Unable to calculate distance for {origin} to {destination}. Setting distance to a default value.")
                distances_matrix[i][j] = 0  # You can set a default value or handle it as needed
        else:
            # Handle the case where i is equal to j (optional, depending on your logic)
            distances_matrix[i][j] = 0  # Or any other appropriate default value


# Appliquer l'algorithme 2-OPT pour optimiser le chemin avec les nouvelles fonctions
optimized_path = two_opt(list(range(len(coord_selected_nodes))), coord_selected_nodes, distances_matrix)

# Afficher le chemin optimisé avec les informations
print("Chemin le plus court:")
total_distance, total_duration = calculate_total_distance(optimized_path, coord_selected_nodes)

coord_2_opt = []
for i in optimized_path:
    point = coord_selected_nodes[i]
    address = get_addresses_for_points([point])[0]
    print(f"- {address} | Coordonnées : {point}")
    coord_2_opt.append(point)

print(f"Distance totale = {total_distance :.2f} m")
print(f"Temps total = {total_duration // 3600} heures {total_duration % 3600 // 60} minutes {total_duration % 60} secondes")

print(coord_2_opt)
print(coord_selected_nodes)

Chemin le plus court:
- Boulevard de Metz, Wazemmes, Lille, Nord, Hauts-de-France, France métropolitaine, 59037, France | Coordonnées : (50.6195125, 3.0410118)
- 7, Place Barthélémy Dorez, Wazemmes, Lille, Nord, Hauts-de-France, France métropolitaine, 59000, France | Coordonnées : (50.6185623, 3.0504404)
- Rue d'Iéna, Wazemmes, Lille, Nord, Hauts-de-France, France métropolitaine, 59046, France | Coordonnées : (50.6232316, 3.0537465)
- 135, Rue Masséna, Wazemmes, Lille, Nord, Hauts-de-France, France métropolitaine, 59800, France | Coordonnées : (50.6295964, 3.0550078)
- Place du Maréchal Leclerc, Wazemmes, Lille, Nord, Hauts-de-France, France métropolitaine, 59037, France | Coordonnées : (50.6289527, 3.0436232)
Distance totale = 3381.20 m
Temps total = 0.0 heures 8.0 minutes 34.700000000000045 secondes
[(50.6195125, 3.0410118), (50.6185623, 3.0504404), (50.6232316, 3.0537465), (50.6295964, 3.0550078), (50.6289527, 3.0436232)]
[(50.6195125, 3.0410118), (50.6185623, 3.0504404), (50.629596

In [7]:
import webbrowser

def open_google_maps(waypoints):
    base_url = "https://www.google.com/maps/dir/"

    # Ajouter les coordonnées des points comme points de passage à l'URL
    for point in waypoints:
        base_url += f"{point[0]},{point[1]}/"

    # Ouvrir l'URL dans le navigateur web par défaut
    webbrowser.open(base_url)

# Liste des points au format (latitude, longitude)
waypoints_list = coord_2_opt

# Ouvrir Google Maps avec les points de passage
open_google_maps(waypoints_list)
